Before answering this question, let's see where we are in the data processing:

![Feature Engineering](assets/feature_engineering.png)

As the diagram above shows, with this checkpoint, we start to mention about how to convert text data into numerical form. This is the **feature engineering** step we need to do before feeding our data into any machine learning algorithm. In the NLP jargon, converting text into numerical form is often called **language modeling** and it's one of the most active research areas in NLP literature. The reason behind why it's called language modeling lies behind semantic. A good numerical representation of the text should be able to capture the meaning of the words and their relationships between each other. We'll talk about the semantic later in this module. For now, we'll just focus on a simple way of representing words in numerical form.

To accommodate the feature generation techniques we learn here and see their performances on a machine learning task, we'll feed our new numerical features into some machine learning algorithms and make some classifications to demonstrate the pros and cons of our feature engineering methods. Hence, our first hands-on NLP application of this module will be the **text classification**. 

Let's start!

# Bag of words (BoW)

Our first feature generation approach will be something called **Bag of Words (BoW)**. BoW is quite simple: our goal is to create a feature matrix such that the rows are observations and there is a column for each unique word in our vocabulary. Then we'll fill in this matrix by counting how many times each word appears in each observation. We will then use those **counts as features**. 

As we said, BoW is simple and very easy to implement using libraries like scikit-learn. In the jargon of scikit-learn, generating BoW features is called **CountVectorizer** as we'll see shortly. However, before moving on to implement the BoW approach, we'll need to do some data cleaning. 

We begin by importing the libraries we'll be using:

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
from nltk.corpus import gutenberg
import nltk
import warnings
warnings.filterwarnings("ignore")

nltk.download('gutenberg')
!python -m spacy download en

Now, we write a helper function called **text_clearner** for cleaning the text. Specifically, we remove some punctuations and numbers from the text:

In [ ]:
# utility function for standard text cleaning
def text_cleaner(text):
    # visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

Now, we load the Persuasion and Alice's Adventures in Wonderland novels of Jane Austen and Lewis Carroll from the Gutenberg module of NLTK. In this checkpoint, we'll be working on these two texts and our ultimate goal will be to distinguish the authors from their sentences. Hence, **our unit of observation will be the sentences of these novels** or in other terms, our documents will be the sentences.

After we load the novels, we do some data cleaning. First, we removed the chapter indicators from the novels and then we apply our text_cleaner function above to clean up some punctuations and the numbers:

In [ ]:
# load and clean the data
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# the Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

Our cleaned texts are stored in two variables called **alice** and **persuasion**. Note that we didn't split the texts into sentences so far. We'll do that using SpaCy. For that purpose, we first load the English module of SpaCy and we parse both alice and persuasion texts with SpaCy:

In [ ]:
# parse the cleaned novels. this can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

We can split our texts into sentences now. You can see how that is easy using SpaCy. Since previously we parsed our documents with SpaCy, we can now use SpaCy's functionalities. In this case, SpaCy will take care of deriving the sentences from the texts. What we need to do is to iterate over the parsed documents after calling the `.sents` attribute. In the following, we iterate using the list comprehension:

In [ ]:
# group into sentences
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# combine the sentences from the two novels into one data frame
sentences = pd.DataFrame(alice_sents + persuasion_sents, columns = ["text", "author"])
sentences.head()

As a result, we ended up having a dataset that consists of two columns. In the first column, we have the sentences and in the second column, we have the authors. Before jumping in the BoW, we need to remove stopwords and punctuations. Then we should convert our tokens to lemmas or stems. In this example, we prefer to lemmatize our tokens. Again, we'll make use of the attributes of the parsed documents by SpaCy:

In [ ]:
# get rid off stop words and punctuation
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = " ".join(
        [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])

Now we can start on converting the text in the first column of our dataset into a numerical form. As we said before, we'll use BoW approach. For this purpose, we'll use `CountVectorizer` from scikit-learn as follows:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word')
X = vectorizer.fit_transform(sentences["text"])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([bow_df, sentences[["text", "author"]]], axis=1)

And that's all. Let's check out our new dataset:

In [ ]:
sentences.head()

As you can see, we have a dataset that we're familiar from the rest of the bootcamp. It's in tabular form: observations sit on rows and we have our features as columns. More importantly, we converted text into a numerical form so that we can apply machine learning algorithms using these as input. This enables us to move to the modeling phase as demonstrated below:

![Modeling](assets/modeling.png)

## BoW in action

Now let's give the bag of words features a whirl by trying some machine learning algorithms:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Y = sentences['author']
X = np.array(sentences.drop(['text','author'], 1))

# We split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

It looks like logistic regression and random forest overfit. Overfitting is a known problem when using bag of words, since it involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected. On the other hand, gradient boosting's training and test scores are close to each other.

# N-grams: words in context

Consider the word ‘vain’ in these two sentences:

    “She labored in vain, the rock would not move.” 

    “She was so vain, her bathroom mirror was covered in lip prints.”

In both sentences, ‘vain’ is an adjective. In sentence 1, it signals a lack of success. In sentence 2, the same word means vanity. Since the two usages can’t be distinguished by their part of speech, how can we tell them apart?

N-grams incorporate context information by creating features made up of a series of consecutive words. The ‘N’ refers to the number of words included in the series. For example, the 2-gram representation of sentence 1 would be:

    (She labored), (labored in), (in vain), (vain the), (the rock), (rock would), (would not), (not move).

The 3-gram representation of sentence 2 would be:

    (She was so), (was so vain), (so vain her), (vain her bathroom), (her bathroom mirror), (bathroom mirror was), (mirror was covered), (was covered in), (covered in lip), (in lip prints).

Each of the word sets could then operate as its own feature. N-grams can be used to create term-document matrices (though it would now be ngram-document matrices), or used in topic modeling. In addition, n-grams are useful for text prediction as they can be used to determine what words are most likely to follow in a sentence, phrase, or search query.

For a sentence with X words, there will be $X-(N-1)$ n-grams. 2-gram phrases are also called ‘bigrams,’ 3-gram phrases are called ‘trigrams,’ etc.

## Why use words alone?

Given the benefits of incorporating word context for distinguishing between different meanings of a word, why would any NLP practitioner worth their salt ever use simple word features? Because models based on single words have several advantages:

* First, n-gram models are considerably more sparse than single-word models. The two ‘vain’ sentences above share four words (‘she’, ‘in’, ‘vain’, ‘the’) but zero n-grams. Sparseness does mean that an n-gram model can be stored in a more memory-efficient way (for example, in a dict that only lists the n-grams that are present in each sentence rather than a set of columns with 1 if an n-gram is present and 0 otherwise). However, it also means that a larger corpus may be needed to detect any shared patterns across documents. In other words, n-gram models may need more documents before they start to give good results.

* Second, single-word models are straightforward to implement, while models incorporating n-grams are more sensitive to fine distinctions of meaning. Which to choose depends on the goals of the NLP project and the trade-offs in time and performance for the specific corpus we are modeling.

## 2-gram example

Implementing n-grams is quite easy using scikit-learn's `CountVectorizer`. The only thing we should do is to give a tuple of range as values to a parameter of `CountVectorizer` which is called `ngram_range`. As the code below demonstrates, we provide a value for the parameter `ngram_range=(2,2)` inside `CountVectorizer`. This means that the vectorizer will produce 2-gram features. If we were to give `ngram_range=(1,2)` as value, then the vectorizer would produce both 1-gram and 2-gram together. However, we keep it as an assignment for you.

Let's generate our 2-grams and see what it looks like:

In [ ]:
# we'll use 2-grams
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,2))
X = vectorizer.fit_transform(sentences["text"])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([bow_df, sentences[["text", "author"]]], axis=1)
sentences.head()

As you see, our new features are 2-gram. Next, let's build the same machine learning models that we built before for the 1-gram case but this time using 2-gram as our features:

In [ ]:
Y = sentences['author']
X = np.array(sentences.drop(['text','author'], 1))

# We split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

The results seem worse than 1-gram! Even overfitting in the logistic regression and the random forest is higher than before. That's because in the 2-gram case, we have more number of features than we have in 1-gram. One possible solution to increase the performance of the models is using 1-gram and 2-gram together as features. This will be one of your tasks in the assignments.